In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display


# Paths & file loading

METRICS_DIR = Path(
    r"E:\2025 fall\Fundamentals of Digital Image Processing\YOLO11\runs\test_metrics"
)

txt_files = sorted(METRICS_DIR.glob("*.txt"))
print("Found txt files:")
for f in txt_files:
    print(" -", f.name)



# Parser for one txt file

def parse_metrics_from_txt(path: Path) -> dict:
    """
    Parse one metrics txt file like:

    AP@[.5:.95](B): 0.767013
    AP@0.5(B)     : 0.902827
    AP@0.75(B)    : 0.830418
    mean P(B)     : 0.865376
    mean R(B)     : 0.910000
    mean F1(B)    : 0.887127
    """
    metrics = {
        "AP":    None,
        "AP50":  None,
        "AP75":  None,
        "P":     None,
        "R":     None,
        "F1":    None,
    }

    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            line_no_space = line.replace(" ", "")

            if "AP@[.5:.95]" in line_no_space or "mAP@[.5:.95]" in line_no_space:
                metrics["AP"] = float(line.split()[-1])
            elif "AP@0.5" in line_no_space:
                metrics["AP50"] = float(line.split()[-1])
            elif "AP@0.75" in line_no_space:
                metrics["AP75"] = float(line.split()[-1])
            elif "meanP(B)" in line_no_space:
                metrics["P"] = float(line.split()[-1])
            elif "meanR(B)" in line_no_space:
                metrics["R"] = float(line.split()[-1])
            elif "meanF1(B)" in line_no_space:
                metrics["F1"] = float(line.split()[-1])

    return metrics



# keep 5 baseline

IP_KEYWORDS = (
    "finetune", "finetuned", "fine_tune", "fine tune",
    "traditional", "fbcnn", "nafnet", "zerodce"
)

def base_condition_from_filename(path: Path):

    name = path.stem.lower()

    if any(k in name for k in IP_KEYWORDS):
        return None

    if "test_d" in name:
        return "D"
    if "raw" in name:
        return "D'"
    if "low" in name and "resolution" in name:
        return "Low Resolution"
    if "motion" in name and "blur" in name:
        return "Motion Blur"
    if "low" in name and "light" in name:
        return "Low Light"

    return None


rows_global = []
for path in txt_files:
    cond = base_condition_from_filename(path)
    if cond is None:
        continue
    m = parse_metrics_from_txt(path)
    m["Condition"] = cond
    rows_global.append(m)

df_global = pd.DataFrame(rows_global)

order = ["D", "D'", "Low Resolution", "Motion Blur", "Low Light"]
df_global["order_idx"] = df_global["Condition"].apply(
    lambda c: order.index(c) if c in order else len(order) + 1
)
df_global = (
    df_global.sort_values("order_idx")
             .drop(columns=["order_idx"])
             .reset_index(drop=True)
)

print("\n=== Global comparison (baseline only) ===")
display(df_global)



# Figure 1: AP / AP50 / AP75

plt.figure(figsize=(8, 4))

x_pos = range(len(df_global))
x_labels = df_global["Condition"]

plt.plot(x_pos, df_global["AP"],   marker="o", label="AP@[.5:.95]")
plt.plot(x_pos, df_global["AP50"], marker="o", label="AP@0.5")
plt.plot(x_pos, df_global["AP75"], marker="o", label="AP@0.75")

plt.xticks(ticks=x_pos, labels=x_labels, rotation=20)
plt.xlabel("Condition")
plt.ylabel("Average Precision")
plt.title("AP metrics across different conditions")
plt.ylim(0, 1.05)
plt.grid(axis="y", linestyle="--", alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


# Figure 2: P / R / F1 

plt.figure(figsize=(8, 4))

bar_width = 0.25
x_pos = range(len(df_global))

plt.bar([x - bar_width for x in x_pos],
        df_global["P"],
        width=bar_width,
        label="P")
plt.bar(x_pos,
        df_global["R"],
        width=bar_width,
        label="R")
plt.bar([x + bar_width for x in x_pos],
        df_global["F1"],
        width=bar_width,
        label="F1")

plt.xticks(ticks=x_pos, labels=df_global["Condition"], rotation=20)
plt.xlabel("Condition")
plt.ylabel("Score")
plt.title("Precision / Recall / F1 across different conditions")
plt.ylim(0, 1.05)
plt.grid(axis="y", linestyle="--", alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

METRICS_DIR = Path(
    r"E:\2025 fall\Fundamentals of Digital Image Processing\YOLO11\runs\test_metrics"
)


def parse_metrics_from_txt(path: Path) -> dict:
    """Parse one YOLO metrics txt file into a dict."""
    metrics = {
        "AP":    None,
        "AP50":  None,
        "AP75":  None,
        "P":     None,
        "R":     None,
        "F1":    None,
    }

    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            no_space = line.replace(" ", "")

            if "AP@[.5:.95]" in no_space or "mAP@[.5:.95]" in no_space:
                metrics["AP"] = float(line.split()[-1])
            elif "AP@0.5" in no_space:
                metrics["AP50"] = float(line.split()[-1])
            elif "AP@0.75" in no_space:
                metrics["AP75"] = float(line.split()[-1])
            elif "mean P(B)" in line:
                metrics["P"] = float(line.split()[-1])
            elif "mean R(B)" in line:
                metrics["R"] = float(line.split()[-1])
            elif "mean F1(B)" in line:
                metrics["F1"] = float(line.split()[-1])

    return metrics


def add_bar_labels(ax, xs, ys, fmt="%.2f", dy=0.015, fontsize=7):
    """Add small numeric labels above bars."""
    for x, y in zip(xs, ys):
        if y is None or np.isnan(y):
            continue
        ax.text(
            x,
            y + dy,
            fmt % y,
            ha="center",
            va="bottom",
            fontsize=fontsize,
        )


# mapping: each test set has distorted D' + traditional IP + ML model
group_files = {
    "Low Resolution": {
        "D'":          "metrics_test_low resolution.txt",
        "Traditional": "metrics_test_low resolution_traditional.txt",
        "FBCNN":       "metrics_test_low resolution_FBCNN.txt",
    },
    "Motion Blur": {
        "D'":          "metrics_test_motion blur.txt",
        "Traditional": "metrics_test_motion blur_traditional.txt",
        "NAFNet":      "metrics_test_motion blur_NAFNet.txt",
    },
    "Low Light": {
        "D'":          "metrics_test_low light.txt",
        "Traditional": "metrics_test_low light_traditional.txt",
        "ZeroDCE":     "metrics_test_low light_ZeroDCE.txt",
    },
}

ap_data  = {k: {} for k in group_files}  # (AP, AP50, AP75)
prf_data = {k: {} for k in group_files}  # (P, R, F1)

for test_name, variants in group_files.items():
    for method_label, fname in variants.items():
        path = METRICS_DIR / fname
        if not path.exists():
            print(f"[Warning] {path} not found, skip.")
            continue

        m = parse_metrics_from_txt(path)
        ap_data[test_name][method_label]  = (m["AP"], m["AP50"], m["AP75"])
        prf_data[test_name][method_label] = (m["P"], m["R"], m["F1"])
        print(
            f"[OK] {test_name} - {method_label}: "
            f"AP={m['AP']:.3f}, AP50={m['AP50']:.3f}, AP75={m['AP75']:.3f}, "
            f"P={m['P']:.3f}, R={m['R']:.3f}, F1={m['F1']:.3f}"
        )


# Figure 1: AP / AP50 / AP75
ap_metric_labels = ["AP@[.5:.95]", "AP@0.5", "AP@0.75"]
ap_colors        = ["#1f77b4", "#ff7f0e", "#2ca02c"]  # blue / orange / green
width = 0.22

fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharey=True)

for ax, (test_name, variants) in zip(axes, group_files.items()):
    method_names = list(variants.keys())
    x = np.arange(len(method_names))

    for k, (label, color) in enumerate(zip(ap_metric_labels, ap_colors)):
        vals = []
        xs = []
        for idx, method in enumerate(method_names):
            AP, AP50, AP75 = ap_data[test_name].get(
                method, (np.nan, np.nan, np.nan)
            )
            if k == 0:
                vals.append(AP)
            elif k == 1:
                vals.append(AP50)
            else:
                vals.append(AP75)
            xs.append(x[idx] + (k - 1) * width)

        ax.bar(
            xs,
            vals,
            width=width,
            color=color,
            label=label if test_name == "Low Resolution" else None,
        )
        add_bar_labels(ax, xs, vals, fmt="%.2f", dy=0.015, fontsize=7)

    ax.set_xticks(x)
    ax.set_xticklabels(method_names, rotation=20)
    ax.set_ylim(0, 1.05)
    ax.set_title(test_name)
    ax.grid(axis="y", linestyle="--", alpha=0.3)

fig.supylabel("Average Precision")
fig.suptitle("AP metrics: D' vs Traditional vs ML")

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper right")
plt.tight_layout(rect=[0, 0, 0.98, 0.92])
plt.show()


# Figure 2: P / R / F1 
fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharey=True)

metric_labels = ["P", "R", "F1"]
metric_colors = {"P": "#1f77b4", "R": "#ff7f0e", "F1": "#2ca02c"}
width = 0.22

for ax, (test_name, variants) in zip(axes, group_files.items()):
    method_names = list(variants.keys())
    x = np.arange(len(method_names))

    for j, metric in enumerate(metric_labels):
        vals = []
        xs = []
        for idx, method in enumerate(method_names):
            P, R, F1 = prf_data[test_name].get(
                method, (np.nan, np.nan, np.nan)
            )
            if metric == "P":
                vals.append(P)
            elif metric == "R":
                vals.append(R)
            else:
                vals.append(F1)
            xs.append(x[idx] + (j - 1) * width)

        ax.bar(
            xs,
            vals,
            width=width,
            color=metric_colors[metric],
            label=metric if test_name == "Low Resolution" else None,
        )
        add_bar_labels(ax, xs, vals, fmt="%.2f", dy=0.015, fontsize=7)

    ax.set_xticks(x)
    ax.set_xticklabels(method_names, rotation=20)
    ax.set_ylim(0, 1.05)
    ax.set_title(test_name)
    ax.grid(axis="y", linestyle="--", alpha=0.3)

fig.supylabel("Score")
fig.suptitle("Precision / Recall / F1: D' vs Traditional vs ML")

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper right")
plt.tight_layout(rect=[0, 0, 0.98, 0.92])
plt.show()


In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt


# Paths to finetune folders

BASE_DIR = Path(r"E:\2025 fall\Fundamentals of Digital Image Processing\YOLO11")

finetune_folders = {
    "Low Light":      BASE_DIR / "finetune_low light",
    "Low Resolution": BASE_DIR / "finetune_low resolution",
    "Motion Blur":    BASE_DIR / "finetune_motion blur",
}


# Helper: load train loss from results.csv

def load_train_loss(csv_path: Path):
    """
    Load training loss curve from a YOLO results.csv.

    Priority:
    1) If column 'train/loss' exists, use it directly.
    2) Otherwise, sum all columns that contain both 'train' and 'loss'.
       This works for formats like 'train/box_loss', 'train/cls_loss', etc.
    """
    df = pd.read_csv(csv_path)

    # x-axis: epoch if available, otherwise just use row index
    if "epoch" in df.columns:
        x = df["epoch"].values
    else:
        x = range(len(df))

    # choose loss column(s)
    if "train/loss" in df.columns:
        loss = df["train/loss"].values
    else:
        loss_cols = [c for c in df.columns if ("train" in c and "loss" in c)]
        if not loss_cols:
            raise ValueError(f"No train loss columns found in {csv_path}.\n"
                             f"Available columns: {list(df.columns)}")
        loss = df[loss_cols].sum(axis=1).values

    return x, loss


# Collect curves for each finetune run

curves = {}

for label, folder in finetune_folders.items():
    csv_path = folder / "results.csv"
    if not csv_path.exists():
        print(f"[Warning] {csv_path} not found, skip.")
        continue

    x, loss = load_train_loss(csv_path)
    curves[label] = (x, loss)
    print(f"[OK] Loaded {label} from {csv_path}")


# Plot all three curves in one figure

plt.figure(figsize=(8, 4))

colors = {
    "Low Light":      "#1f77b4",  # blue
    "Low Resolution": "#ff7f0e",  # orange
    "Motion Blur":    "#2ca02c",  # green
}

for label, (x, loss) in curves.items():
    plt.plot(x, loss, label=label,
             linewidth=2,
             marker="o",
             markersize=4,
             color=colors.get(label, None))

plt.xlabel("Epoch")
plt.ylabel("Training loss")
plt.title("Training loss curves for fine-tuned models")
plt.grid(True, linestyle="--", alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch


# Paths & file loading

METRICS_DIR = Path(
    r"E:\2025 fall\Fundamentals of Digital Image Processing\YOLO11\runs\test_metrics"
)

txt_files = sorted(METRICS_DIR.glob("*.txt"))
print("Found txt files:")
for f in txt_files:
    print(" -", f.name)


# Helper functions

def detect_testset(name_low: str):
    """Decide which test set this file belongs to based on its filename."""
    if "low resolution" in name_low:
        return "Low Resolution"
    if "motion blur" in name_low:
        return "Motion Blur"
    if "low light" in name_low:
        return "Low Light"
    return None


def detect_variant(name_low: str):
    """
    Decide which variant it is, based on its filename:
      - Fine-Tuned      (pure finetuning, no extra IP method)
      - Traditional     (contains 'traditional')
      - ML              (NAFNet / FBCNN / ZeroDCE etc.)
    """
    # Traditional IP methods
    if "traditional" in name_low:
        return "Traditional"

    # Learning-based IP methods
    if ("nafnet" in name_low) or ("fbcnn" in name_low) or ("zerodce" in name_low):
        return "ML"

    # Pure finetuning (no IP keyword)
    if ("finetune" in name_low) or ("fine tune" in name_low):
        return "Fine-Tuned"

    return None


def parse_metrics_from_txt(path: Path):
    """
    Parse one metrics txt file, e.g.

    AP@[.5:.95](B): 0.767013
    AP@0.5(B)     : 0.902827
    AP@0.75(B)    : 0.830418
    mean P(B)     : 0.865376
    mean R(B)     : 0.910000
    mean F1(B)    : 0.887127
    """
    metrics = {
        "AP":   None,
        "AP50": None,
        "AP75": None,
        "P":    None,
        "R":    None,
        "F1":   None,
    }

    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            no_space = line.replace(" ", "")

            if ("AP@[.5:.95]" in no_space) or ("mAP@[.5:.95]" in no_space):
                metrics["AP"] = float(line.split()[-1])
            elif "AP@0.5" in no_space:
                metrics["AP50"] = float(line.split()[-1])
            elif "AP@0.75" in no_space:
                metrics["AP75"] = float(line.split()[-1])
            elif "meanP(B)" in no_space:
                metrics["P"] = float(line.split()[-1])
            elif "meanR(B)" in no_space:
                metrics["R"] = float(line.split()[-1])
            elif "meanF1(B)" in no_space:
                metrics["F1"] = float(line.split()[-1])

    return metrics


# Small helper: add value labels on top of bars
def add_bar_labels(ax, xs, ys, fmt="%.2f", dy=0.01, fontsize=7):
    """
    Add numeric value labels above bars.

    Args:
        ax: Matplotlib axis.
        xs: x positions of bars.
        ys: heights of bars.
        fmt: format string for values.
        dy: vertical offset to avoid overlap with bar edge.
        fontsize: text font size.
    """
    for x, y in zip(xs, ys):
        if y is None or np.isnan(y):
            continue
        ax.text(
            x,
            y + dy,
            fmt % y,
            ha="center",
            va="bottom",
            fontsize=fontsize,
        )


# Build DataFrame

rows = []

for path in txt_files:
    name_low = path.stem.lower()

    # Keep only finetune-related results
    if ("finetune" not in name_low) and ("fine tune" not in name_low):
        continue

    testset = detect_testset(name_low)
    variant = detect_variant(name_low)

    if (testset is None) or (variant is None):
        continue

    metrics = parse_metrics_from_txt(path)
    metrics["TestSet"] = testset
    metrics["Variant"] = variant
    rows.append(metrics)

df = pd.DataFrame(rows)

test_order = ["Low Resolution", "Motion Blur", "Low Light"]
variant_order = ["Fine-Tuned", "Traditional", "ML"]

df["test_idx"] = df["TestSet"].apply(
    lambda s: test_order.index(s) if s in test_order else 999
)
df["variant_idx"] = df["Variant"].apply(
    lambda s: variant_order.index(s) if s in variant_order else 999
)

df = (
    df.sort_values(["test_idx", "variant_idx"])
      .reset_index(drop=True)
      .drop(columns=["test_idx", "variant_idx"])
)

print("\nParsed metrics for finetune comparison:")
display(df)

# Display name for the ML method on x-axis for each test set
ml_label_map = {
    "Low Resolution": "FBCNN",
    "Motion Blur": "NAFNet",
    "Low Light": "ZeroDCE",
}


# Figure 1: AP / AP@0.5 / AP@0.75 bar plots



ap_metrics_order = ["AP", "AP50", "AP75"]
ap_labels = {
    "AP":   "AP@[.5:.95]",
    "AP50": "AP@0.5",
    "AP75": "AP@0.75",
}
ap_colors = {
    "AP":   "tab:blue",
    "AP50": "tab:orange",
    "AP75": "tab:green",
}

fig, axes = plt.subplots(1, len(test_order), figsize=(12, 4), sharey=True)
bar_width = 0.18  # thinner bars

for i, tset in enumerate(test_order):
    ax = axes[i]
    sub = df[df["TestSet"] == tset].set_index("Variant")

    base_positions = list(range(len(variant_order)))  # 0,1,2

    # Draw three groups of AP-related bars
    for m_idx, metric in enumerate(ap_metrics_order):
        ys = []
        xs = []
        offset = (m_idx - 1) * bar_width  # -1,0,1
        for v_idx, v in enumerate(variant_order):
            if v in sub.index and pd.notna(sub.loc[v, metric]):
                val = sub.loc[v, metric]
                if isinstance(val, pd.Series):
                    val = val.iloc[0]
                ys.append(float(val))
            else:
                ys.append(float("nan"))
            xs.append(base_positions[v_idx] + offset)

        ax.bar(
            xs,
            ys,
            width=bar_width,
            color=ap_colors[metric],
        )
        # Add value labels for this metric
        add_bar_labels(ax, xs, ys, fmt="%.2f", dy=0.015, fontsize=7)

    # X-tick labels: Fine-Tuned / Traditional / specific ML method
    xtick_labels = []
    for v in variant_order:
        if v == "ML":
            xtick_labels.append(ml_label_map.get(tset, "ML"))
        else:
            xtick_labels.append(v)

    ax.set_xticks(base_positions)
    ax.set_xticklabels(xtick_labels, rotation=20)
    ax.set_title(tset)
    if i == 0:
        ax.set_ylabel("Score")
    ax.set_ylim(0, 1.05)
    ax.grid(axis="y", linestyle="--", alpha=0.3)

# Global legend on the top-right
ap_handles = [Patch(color=ap_colors[m], label=ap_labels[m]) for m in ap_metrics_order]
fig.legend(
    handles=ap_handles,
    title="AP Metric",
    loc="upper right",
    bbox_to_anchor=(1, 0.98),
    ncol=1,
    borderaxespad=0.3,
)

fig.suptitle(
    "AP metrics comparison: Fine-Tuning vs IP",
    y=0.96,
)
plt.tight_layout(rect=[0, 0, 1, 0.9])
plt.show()


# Figure 2: P / R / F1 bar plots

metrics_order = ["P", "R", "F1"]
metrics_colors = {
    "P": "tab:blue",
    "R": "tab:orange",
    "F1": "tab:green",
}

fig, axes = plt.subplots(1, len(test_order), figsize=(12, 4), sharey=True)
bar_width = 0.18  # thinner bars

for i, tset in enumerate(test_order):
    ax = axes[i]
    sub = df[df["TestSet"] == tset].set_index("Variant")

    base_positions = list(range(len(variant_order)))

    for m_idx, metric in enumerate(metrics_order):
        ys = []
        xs = []
        offset = (m_idx - 1) * bar_width  # -1,0,1
        for v_idx, v in enumerate(variant_order):
            if v in sub.index and pd.notna(sub.loc[v, metric]):
                val = sub.loc[v, metric]
                if isinstance(val, pd.Series):
                    val = val.iloc[0]
                ys.append(float(val))
            else:
                ys.append(float("nan"))
            xs.append(base_positions[v_idx] + offset)

        ax.bar(
            xs,
            ys,
            width=bar_width,
            color=metrics_colors[metric],
        )
        # Add value labels for this metric
        add_bar_labels(ax, xs, ys, fmt="%.2f", dy=0.015, fontsize=7)

    # X-axis labels: replace "ML" with actual method name
    xtick_labels = []
    for v in variant_order:
        if v == "ML":
            xtick_labels.append(ml_label_map.get(tset, "ML"))
        else:
            xtick_labels.append(v)

    ax.set_xticks(base_positions)
    ax.set_xticklabels(xtick_labels, rotation=20)
    ax.set_title(tset)
    if i == 0:
        ax.set_ylabel("Score")
    ax.set_ylim(0, 1.05)
    ax.grid(axis="y", linestyle="--", alpha=0.3)

# Global legend for P/R/F1
handles = [Patch(color=metrics_colors[m], label=m) for m in metrics_order]
fig.legend(
    handles=handles,
    title="Metric",
    loc="upper right",
    bbox_to_anchor=(0.98, 0.98),
    ncol=1,
    borderaxespad=0.3,
)

fig.suptitle(
    "P / R / F1 comparison: Fine-Tuning vs IP",
    y=0.96,
)
plt.tight_layout(rect=[0, 0, 1, 0.9])
plt.show()
